In [1]:
import os
import pandas as pd
import numpy as np

import datetime as dt
import sqlite3

from scipy.stats.mstats import winsorize
from scipy.stats.mstats import spearmanr
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#import talib 

# Model packages
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA

# K-Fold Cross Validation
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
snp = pd.read_csv(r"C:\Users\suraj\Desktop\Brunel MPhil\SP500 Prediction_v1.1.py_Arkaja\Datasets_new\SP500FUT.csv")
snp

,date,open,low,high,close,RSI,BBANDS_upper,BBANDS_middle,BBANDS_lower,MOM,...,ROCR,ROCR100,STOCH_slowk,STOCH_slowd,STOCHF_fastk,STOCHF_fastd,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR
0,20050103,1213.50,1201.75,1221.25,1206.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20050104,1206.25,1186.75,1209.50,1191.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20050105,1190.75,1182.75,1195.00,1183.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20050106,1183.25,1182.75,1193.75,1188.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20050107,1188.50,1183.50,1194.50,1186.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530,20221223,3850.00,3821.25,3872.50,3869.75,44.849514,4035.060544,3920.265714,3805.470885,-133.50,...,0.966652,96.665210,52.548780,38.059895,61.904762,52.548780,59.912507,62.134944,46.887794,-77.208976
4531,20221227,3878.00,3837.25,3900.50,3855.00,43.596236,4032.390015,3913.837143,3795.284271,-90.00,...,0.977186,97.718631,52.952381,47.691216,50.666667,52.952381,36.070417,40.825083,50.586545,-81.346424
4532,20221228,3858.00,3804.50,3875.00,3807.50,39.744530,4034.198955,3904.605000,3775.011045,-129.25,...,0.967168,96.716835,42.349206,49.283456,14.476190,42.349206,0.000000,31.994308,46.380547,-94.670407
4533,20221229,3811.00,3806.25,3882.75,3871.75,46.615016,4023.805281,3897.890714,3771.976147,-94.00,...,0.976297,97.629704,42.857143,46.052910,63.428571,42.857143,100.000000,45.356806,51.655508,-76.647966


In [3]:
# Setting 'datetime' format to date
snp['year'] = (snp['date']/10000).astype(int)
snp['month'] = ((snp['date']%10000)/100).astype(int)
snp['day'] = (snp['date']%100).astype(int)
snp

,date,open,low,high,close,RSI,BBANDS_upper,BBANDS_middle,BBANDS_lower,MOM,...,STOCH_slowd,STOCHF_fastk,STOCHF_fastd,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR,year,month,day
0,20050103,1213.50,1201.75,1221.25,1206.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,3
1,20050104,1206.25,1186.75,1209.50,1191.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,4
2,20050105,1190.75,1182.75,1195.00,1183.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,5
3,20050106,1183.25,1182.75,1193.75,1188.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,6
4,20050107,1188.50,1183.50,1194.50,1186.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530,20221223,3850.00,3821.25,3872.50,3869.75,44.849514,4035.060544,3920.265714,3805.470885,-133.50,...,38.059895,61.904762,52.548780,59.912507,62.134944,46.887794,-77.208976,2022,12,23
4531,20221227,3878.00,3837.25,3900.50,3855.00,43.596236,4032.390015,3913.837143,3795.284271,-90.00,...,47.691216,50.666667,52.952381,36.070417,40.825083,50.586545,-81.346424,2022,12,27
4532,20221228,3858.00,3804.50,3875.00,3807.50,39.744530,4034.198955,3904.605000,3775.011045,-129.25,...,49.283456,14.476190,42.349206,0.000000,31.994308,46.380547,-94.670407,2022,12,28
4533,20221229,3811.00,3806.25,3882.75,3871.75,46.615016,4023.805281,3897.890714,3771.976147,-94.00,...,46.052910,63.428571,42.857143,100.000000,45.356806,51.655508,-76.647966,2022,12,29


In [4]:
snp['date_2'] = snp[['year','month','day']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
snp['date_2'] = pd.to_datetime(snp['date_2'])
snp

,date,open,low,high,close,RSI,BBANDS_upper,BBANDS_middle,BBANDS_lower,MOM,...,STOCHF_fastk,STOCHF_fastd,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR,year,month,day,date_2
0,20050103,1213.50,1201.75,1221.25,1206.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,3,2005-01-03
1,20050104,1206.25,1186.75,1209.50,1191.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,4,2005-01-04
2,20050105,1190.75,1182.75,1195.00,1183.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,5,2005-01-05
3,20050106,1183.25,1182.75,1193.75,1188.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,6,2005-01-06
4,20050107,1188.50,1183.50,1194.50,1186.25,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005,1,7,2005-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530,20221223,3850.00,3821.25,3872.50,3869.75,44.849514,4035.060544,3920.265714,3805.470885,-133.50,...,61.904762,52.548780,59.912507,62.134944,46.887794,-77.208976,2022,12,23,2022-12-23
4531,20221227,3878.00,3837.25,3900.50,3855.00,43.596236,4032.390015,3913.837143,3795.284271,-90.00,...,50.666667,52.952381,36.070417,40.825083,50.586545,-81.346424,2022,12,27,2022-12-27
4532,20221228,3858.00,3804.50,3875.00,3807.50,39.744530,4034.198955,3904.605000,3775.011045,-129.25,...,14.476190,42.349206,0.000000,31.994308,46.380547,-94.670407,2022,12,28,2022-12-28
4533,20221229,3811.00,3806.25,3882.75,3871.75,46.615016,4023.805281,3897.890714,3771.976147,-94.00,...,63.428571,42.857143,100.000000,45.356806,51.655508,-76.647966,2022,12,29,2022-12-29


In [5]:
snp = snp.drop(columns=['date', 'year','month','day'])
snp.rename(columns={"date_2": "date"}, inplace=True)
snp = snp.sort_values(by='date', ascending=True)
snp.head()

,open,low,high,close,RSI,BBANDS_upper,BBANDS_middle,BBANDS_lower,MOM,ADXR,...,ROCR100,STOCH_slowk,STOCH_slowd,STOCHF_fastk,STOCHF_fastd,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR,date
0,1213.50,1201.75,1221.25,1206.25,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005-01-03
1,1206.25,1186.75,1209.50,1191.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005-01-04
2,1190.75,1182.75,1195.00,1183.25,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005-01-05
3,1183.25,1182.75,1193.75,1188.25,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005-01-06
4,1188.50,1183.50,1194.50,1186.25,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005-01-07


In [6]:
# Close to close estimation of UP_DOWN
snp['up_down'] = (snp['close'].shift(-1)>snp['close']).astype(int)
snp = snp[:-1]  # We need to drop the last observation of the df
snp

,open,low,high,close,RSI,BBANDS_upper,BBANDS_middle,BBANDS_lower,MOM,ADXR,...,STOCH_slowk,STOCH_slowd,STOCHF_fastk,STOCHF_fastd,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR,date,up_down
0,1213.50,1201.75,1221.25,1206.25,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005-01-03,0
1,1206.25,1186.75,1209.50,1191.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005-01-04,0
2,1190.75,1182.75,1195.00,1183.25,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005-01-05,1
3,1183.25,1182.75,1193.75,1188.25,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005-01-06,0
4,1188.50,1183.50,1194.50,1186.25,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005-01-07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4529,3915.00,3788.50,3919.75,3849.25,42.724589,4048.344997,3929.801429,3811.257860,-226.25,12.736704,...,37.572487,24.148686,46.285714,37.572487,26.492325,43.234637,41.584694,-82.959327,2022-12-22,1
4530,3850.00,3821.25,3872.50,3869.75,44.849514,4035.060544,3920.265714,3805.470885,-133.50,12.454398,...,52.548780,38.059895,61.904762,52.548780,59.912507,62.134944,46.887794,-77.208976,2022-12-23,0
4531,3878.00,3837.25,3900.50,3855.00,43.596236,4032.390015,3913.837143,3795.284271,-90.00,12.030512,...,52.952381,47.691216,50.666667,52.952381,36.070417,40.825083,50.586545,-81.346424,2022-12-27,0
4532,3858.00,3804.50,3875.00,3807.50,39.744530,4034.198955,3904.605000,3775.011045,-129.25,11.723592,...,42.349206,49.283456,14.476190,42.349206,0.000000,31.994308,46.380547,-94.670407,2022-12-28,1


In [7]:
# Generating lagged value of 'up_down' and its moving average
snp['updown_lag'] = snp['up_down'].shift(1) #should it be -1 ?
snp['updown_lag_MA3'] = snp['updown_lag'].rolling(3).mean()

snp['ret'] = snp['close'].pct_change()

snp = snp.drop(columns=['open', 'low', 'high', 'close'])

snp = snp.dropna()
snp.head()

,RSI,BBANDS_upper,BBANDS_middle,BBANDS_lower,MOM,ADXR,APO,AROON_down,AROON_up,AROONOSC,...,STOCHF_fastd,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR,date,up_down,updown_lag,updown_lag_MA3,ret
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2005-01-06,0,1.0,0.333333,0.004226
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2005-01-07,1,0.0,0.333333,-0.001683
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2005-01-10,0,1.0,0.666667,0.004636
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.245256,0.0,0.0,0.0,0.0,2005-01-11,1,0.0,0.333333,-0.006503
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.131620,0.0,0.0,0.0,0.0,2005-01-12,0,1.0,0.666667,0.002956


In [8]:
# Retaining Data until 31 Dec, 2022 only
snp = snp[snp['date']<='2022-12-31']
snp['date'].tail()

4529   2022-12-22
4530   2022-12-23
4531   2022-12-27
4532   2022-12-28
4533   2022-12-29
Name: date, dtype: datetime64[ns]

In [9]:
# Getting penultimate month's last observation
groups = snp.groupby([pd.DatetimeIndex(snp['date']).year, pd.DatetimeIndex(snp['date']).month])
index_grp = []
for idx, group in groups:
    index_grp.append(group.index.max())
    # index_grp[-2] refers to the index of the second last item in the list and refers to last... 
    # trading day of penultimate month

In [10]:
index_grp


[19,
 38,
 60,
 81,
 102,
 124,
 144,
 167,
 188,
 209,
 230,
 251,
 271,
 290,
 313,
 332,
 354,
 376,
 396,
 419,
 439,
 461,
 482,
 502,
 523,
 542,
 564,
 584,
 606,
 627,
 648,
 671,
 690,
 713,
 734,
 754,
 775,
 795,
 815,
 837,
 858,
 879,
 902,
 923,
 944,
 967,
 986,
 1008,
 1028,
 1047,
 1069,
 1090,
 1110,
 1132,
 1154,
 1175,
 1196,
 1218,
 1238,
 1260,
 1279,
 1298,
 1321,
 1342,
 1362,
 1384,
 1405,
 1427,
 1448,
 1469,
 1490,
 1512,
 1532,
 1551,
 1574,
 1594,
 1615,
 1637,
 1657,
 1680,
 1701,
 1722,
 1743,
 1764,
 1784,
 1804,
 1826,
 1846,
 1868,
 1889,
 1910,
 1933,
 1952,
 1973,
 1994,
 2014,
 2035,
 2054,
 2074,
 2096,
 2118,
 2138,
 2160,
 2182,
 2202,
 2225,
 2245,
 2266,
 2287,
 2306,
 2327,
 2348,
 2369,
 2390,
 2412,
 2433,
 2454,
 2477,
 2496,
 2518,
 2538,
 2557,
 2579,
 2601,
 2621,
 2643,
 2665,
 2686,
 2707,
 2729,
 2749,
 2771,
 2790,
 2810,
 2832,
 2853,
 2874,
 2896,
 2916,
 2939,
 2960,
 2981,
 3002,
 3023,
 3043,
 3062,
 3085,
 3104,
 3126,
 3148,
 

In [11]:
# For penultimate month choose k = -2, for 3rd last choose k = -3 and so on...
# For prediction from Jan, 2017 onwards
k = (snp['date'].max().year - dt.datetime(2016, 12, 31).year)*12 + (snp['date'].max().month - dt.datetime(2016, 12, 31).month)
k = (k+1)*(-1)

In [12]:
## Defining dependent and independent-variable dataframes
data_x = snp.drop(columns=['date', 'up_down'])
data_y = snp[['up_down']] # should this not be the lagged ?

In [13]:
# Rolling window (rw = 1000) regression - Prediction last month's "up_down"
rw_dict = {'rw_nb':1000, 'rw_rf':250, 'rw_dst':200, 'rw_ann':250, 'rw_logreg':200}

In [14]:
# Model Declaration
logreg = LogisticRegression(random_state=rw_dict['rw_logreg'])
nb = GaussianNB()
rf = RandomForestClassifier(random_state=rw_dict['rw_rf'])
dst = DecisionTreeClassifier(random_state=rw_dict['rw_dst'])
ann = MLPClassifier(random_state=rw_dict['rw_ann'])


In [15]:
model_dict = {'nb': nb, 'rf':rf, 'dst': dst, 'ann': ann, 'logreg': logreg}

In [16]:
window = 200
n = index_grp[k] 
X_train = data_x.iloc[n-window-1:n+1, :]
X_train
y_train = data_y.iloc[n-window-1:n+1, 0]
y_train
scaler = StandardScaler().fit(X_train)
scaler
X_train_scaled = scaler.transform(X_train)
X_train_scaled
true_data = snp[['date', 'up_down']].iloc[n+1:n+2, :]
true_data

,date,up_down
3027,2017-01-06,0


In [18]:
# Storing predicted values of y
pred_logreg = pd.DataFrame(columns=['date', 'up_down','logreg', 'prob_logreg']) # DataFrame to store LOGREG outcome
pred_nb = pd.DataFrame(columns=['date', 'up_down','nb', 'prob_nb']) # DataFrame to store NB outcome
pred_rf = pd.DataFrame(columns=['date', 'up_down','rf', 'prob_rf']) # DataFrame to store RF outcome
pred_dst = pd.DataFrame(columns=['date', 'up_down','dst', 'prob_dst']) # DataFrame to store DST outcome
pred_ann = pd.DataFrame(columns=['date', 'up_down','ann', 'prob_ann']) # DataFrame to store ANN outcome

pred_dict = {'pred_nb': pred_nb, 'pred_rf': pred_rf,
              'pred_dst': pred_dst, 'pred_ann': pred_ann, 'pred_logreg': pred_logreg}

model = ['nb', 'rf', 'dst', 'ann', 'logreg']
for ls in model:

    ## Finding Index for Models
    n = index_grp[k] 
        # it will take value until the last trading of the penultimate month. 'n' used for splitting in Training model
    N = (snp.shape[0]-index_grp[k]-1) # Number of loops
    print('indices reference', k, n, N)    
    
    for _ in range(0, N):

        window = rw_dict['rw_'+ls]
        
        # Training Model    
        X_train = data_x.iloc[n-window-1:n+1, :]
        y_train = data_y.iloc[n-window-1:n+1, 0]

        # Testing Model
        X_test = data_x.iloc[n+1:n+2, :]
        y_test = data_y.iloc[n+1:n+2, 0]

        # Scaling all independent variables
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Implementing Principal Component Analyses
        if ls == 'nb' or ls=='rf':
            pca = PCA(0.9)
        elif ls == 'dst' or ls=='ann':
            pca = PCA(0.99)
        elif ls == 'logreg':
            pca = PCA(0.95)
            
        pca.fit(X_train)
        train_pca = pca.transform(X_train_scaled)
        test_pca = pca.transform(X_test_scaled)

        # Storing True Data for Y
        true_data = snp[['date', 'up_down']].iloc[n+1:n+2, :]
        true_data = true_data.reset_index().drop(columns=['index'])

        # Model Prediction
        model_predict = model_dict[ls].fit(train_pca, y_train)
        y_pred = model_predict.predict(test_pca)
        y_prob = model_predict.predict_proba(test_pca)

        # Storing NB model prediction
        df_reg = pd.DataFrame({ls: y_pred.tolist(), 'prob_'+ls: y_prob.tolist()})
        df = pd.concat([true_data, df_reg], axis=1)
        pred_dict['pred_'+ls] = pd.concat([pred_dict['pred_'+ls], df])  #Appending predictions to DF

        n += 1

    print("Confusion Matrix of "+ls+" predictions")

    pred_dict['pred_'+ls]['up_down'] = pred_dict['pred_'+ls]['up_down'].astype(float)
    pred_dict['pred_'+ls][ls] = pred_dict['pred_'+ls][ls].astype(float)

    a = confusion_matrix(pred_dict['pred_'+ls]['up_down'], pred_dict['pred_'+ls][ls])
    b = round((a[0][0] + a[1][1])/(a[0][0]+ a[1][1] + a[0][1] + a[1][0]), 4)
    print(a, b)

indices reference -73 3023 1507
Confusion Matrix of nb predictions
[[ 44 643]
 [ 59 761]] 0.5342
indices reference -73 3023 1507
Confusion Matrix of rf predictions
[[316 371]
 [369 451]] 0.509
indices reference -73 3023 1507
Confusion Matrix of dst predictions
[[366 321]
 [387 433]] 0.5302
indices reference -73 3023 1507
Confusion Matrix of ann predictions
[[320 367]
 [354 466]] 0.5216
indices reference -73 3023 1507
Confusion Matrix of logreg predictions
[[181 506]
 [222 598]] 0.5169


In [19]:
pred_dict

{'pred_nb':          date  up_down   nb                                    prob_nb
 0  2017-01-06      0.0  0.0   [0.5342163756850007, 0.4657836243149997]
 0  2017-01-09      0.0  0.0  [0.5375566433621161, 0.46244335663788394]
 0  2017-01-10      1.0  0.0   [0.538513615865299, 0.46148638413470056]
 0  2017-01-11      0.0  0.0   [0.5376800856506206, 0.4623199143493796]
 0  2017-01-12      1.0  0.0  [0.5355701429502123, 0.46442985704978756]
 ..        ...      ...  ...                                        ...
 0  2022-12-22      1.0  1.0   [0.4801085803053846, 0.5198914196946154]
 0  2022-12-23      0.0  1.0   [0.4787503842332725, 0.5212496157667276]
 0  2022-12-27      0.0  1.0  [0.47896563753847154, 0.5210343624615283]
 0  2022-12-28      1.0  1.0   [0.4794615675697241, 0.5205384324302761]
 0  2022-12-29      0.0  1.0   [0.47753549563353814, 0.522464504366462]
 
 [1507 rows x 4 columns],
 'pred_rf':          date  up_down   rf       prob_rf
 0  2017-01-06      0.0  0.0  [0.62, 0.38]


In [20]:
for ls in ['nb', 'rf', 'dst', 'ann', 'logreg']:
    a = confusion_matrix(pred_dict['pred_'+ls]['up_down'], pred_dict['pred_'+ls][ls])
    print(a[0][0]+ a[1][1] + a[0][1] + a[1][0])
    
    pred_dict['pred_'+ls]['date'] = pd.to_datetime(pred_dict['pred_'+ls]['date'], format='%Y-%m-%d')
    pred_dict['pred_'+ls].to_csv("Predictions_and_Probability_{}_v1.1.csv".format(ls), index=False)

1507
1507
1507
1507
1507


In [21]:
output = pd.merge(pred_dict['pred_'+model[0]], pred_dict['pred_'+model[1]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[2]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')

output['sum'] =  output['nb'] + output['rf'] + output['dst'] + output['ann'] + output['logreg']

# Trade is executed only when all 5 models converge
a = output['up_down'][output['sum']>=5].count()  
b = output['up_down'][(output['sum']>=5) & (output['up_down']==1)].count()

c = output['up_down'][output['sum']==0].count()
d = output['up_down'][(output['sum']==0) & (output['up_down']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(pred_dict['pred_'+model[0]]), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', (b+d)/(a+c)*100,'%')

print('UP Accuracy:', (b)/(a)*100,'%')
print('DOWN Accuracy:', (d)/(c)*100,'%')

from sklearn.metrics import confusion_matrix
import numpy as np

# Create the confusion matrix
confusion_matrix = np.array([[d, c-d], [a-b, b]])

print("Confusion Matrix:")
print(confusion_matrix)



Predictions Window-size: 1507 
 Total Predictions Made: 236 
 Correct Predictions: 139
Overall Accuracy: 58.89830508474576 %
UP Accuracy: 59.82142857142857 %
DOWN Accuracy: 41.66666666666667 %
Confusion Matrix:
[[  5   7]
 [ 90 134]]


In [22]:
output = pd.merge(pred_dict['pred_'+model[0]], pred_dict['pred_'+model[1]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[2]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')

output['sum'] =  output['nb'] + output['rf'] + output['dst'] + output['ann'] + output['logreg']

# Trade is executed only when 4 of 5 models converge
a = output['up_down'][output['sum']>=4].count()  
b = output['up_down'][(output['sum']>=4) & (output['up_down']==1)].count()

c = output['up_down'][output['sum']<=1].count()
d = output['up_down'][(output['sum']<=1) & (output['up_down']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(pred_dict['pred_'+model[0]]), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', (b+d)/(a+c)*100,'%')

print('UP Accuracy:', (b)/(a)*100,'%')
print('DOWN Accuracy:', (d)/(c)*100,'%')

# Create the confusion matrix
confusion_matrix = np.array([[d, c-d], [a-b, b]])

print("Confusion Matrix:")
print(confusion_matrix)

Predictions Window-size: 1507 
 Total Predictions Made: 754 
 Correct Predictions: 414
Overall Accuracy: 54.907161803713535 %
UP Accuracy: 56.04229607250756 %
DOWN Accuracy: 46.73913043478261 %
Confusion Matrix:
[[ 43  49]
 [291 371]]


In [23]:
output = pd.merge(pred_dict['pred_'+model[0]], pred_dict['pred_'+model[1]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[2]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')

output['sum'] =  output['nb'] + output['rf'] + output['dst'] + output['ann'] + output['logreg']

# Trade is executed only when 3 of 5 models converge
a = output['up_down'][output['sum']>=3].count()  
b = output['up_down'][(output['sum']>=3) & (output['up_down']==1)].count()

c = output['up_down'][output['sum']<=2].count()
d = output['up_down'][(output['sum']<=2) & (output['up_down']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(pred_dict['pred_'+model[0]]), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', (b+d)/(a+c)*100,'%')

print('UP Accuracy:', (b)/(a)*100,'%')
print('DOWN Accuracy:', (d)/(c)*100,'%')

# Create the confusion matrix
confusion_matrix = np.array([[d, c-d], [a-b, b]])

print("Confusion Matrix:")
print(confusion_matrix)

Predictions Window-size: 1507 
 Total Predictions Made: 1507 
 Correct Predictions: 806
Overall Accuracy: 53.48374253483743 %
UP Accuracy: 55.30776092774309 %
DOWN Accuracy: 48.18652849740933 %
Confusion Matrix:
[[186 200]
 [501 620]]


In [24]:
output = pd.merge(pred_dict['pred_'+model[0]], pred_dict['pred_'+model[2]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')

output['sum'] =  output['nb'] + output['dst'] + output['ann'] 

# Trade is executed only when all three models converge
a = output['up_down'][output['sum']>=3].count()  
b = output['up_down'][(output['sum']>=3) & (output['up_down']==1)].count()

c = output['up_down'][output['sum']<=0].count()
d = output['up_down'][(output['sum']<=0) & (output['up_down']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(pred_dict['pred_'+model[0]]), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', (b+d)/(a+c)*100,'%')

print('UP Accuracy:', (b)/(a)*100,'%')
print('DOWN Accuracy:', (d)/(c)*100,'%')

# Create the confusion matrix
confusion_matrix = np.array([[d, c-d], [a-b, b]])

print("Confusion Matrix:")
print(confusion_matrix)

Predictions Window-size: 1507 
 Total Predictions Made: 436 
 Correct Predictions: 251
Overall Accuracy: 57.56880733944955 %
UP Accuracy: 57.98525798525799 %
DOWN Accuracy: 51.724137931034484 %
Confusion Matrix:
[[ 15  14]
 [171 236]]


In [25]:
output = pd.merge(pred_dict['pred_'+model[0]], pred_dict['pred_'+model[2]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')

output['sum'] =  output['nb'] + output['dst'] + output['ann'] 

# Trade is executed only when any 2 models converge
a = output['up_down'][output['sum']>=2].count()  
b = output['up_down'][(output['sum']>=2) & (output['up_down']==1)].count()

c = output['up_down'][output['sum']<=1].count()
d = output['up_down'][(output['sum']<=1) & (output['up_down']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(pred_dict['pred_'+model[0]]), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', (b+d)/(a+c)*100,'%')

print('UP Accuracy:', (b)/(a)*100,'%')
print('DOWN Accuracy:', (d)/(c)*100,'%')

# Create the confusion matrix
confusion_matrix = np.array([[d, c-d], [a-b, b]])

print("Confusion Matrix:")
print(confusion_matrix)

Predictions Window-size: 1507 
 Total Predictions Made: 1507 
 Correct Predictions: 817
Overall Accuracy: 54.2136695421367 %
UP Accuracy: 55.87703435804702 %
DOWN Accuracy: 49.62593516209476 %
Confusion Matrix:
[[199 202]
 [488 618]]


In [26]:
output = pd.merge(pred_dict['pred_'+model[2]], pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')

output['sum'] =  output['dst'] + output['dst'] + output['logreg'] 

# Trade is executed only when all three models converge
a = output['up_down'][output['sum']>=3].count()  
b = output['up_down'][(output['sum']>=3) & (output['up_down']==1)].count()

c = output['up_down'][output['sum']<=0].count()
d = output['up_down'][(output['sum']<=0) & (output['up_down']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(pred_dict['pred_'+model[0]]), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', (b+d)/(a+c)*100,'%')

print('UP Accuracy:', (b)/(a)*100,'%')
print('DOWN Accuracy:', (d)/(c)*100,'%')

# Create the confusion matrix
confusion_matrix = np.array([[d, c-d], [a-b, b]])

print("Confusion Matrix:")
print(confusion_matrix)

Predictions Window-size: 1507 
 Total Predictions Made: 777 
 Correct Predictions: 424
Overall Accuracy: 54.56885456885456 %
UP Accuracy: 56.91489361702128 %
DOWN Accuracy: 48.35680751173709 %
Confusion Matrix:
[[103 110]
 [243 321]]


In [27]:
output = pd.merge(pred_dict['pred_'+model[2]], pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[3]], on=['date', 'up_down'], how='inner')
#output = output.merge(pred_dict['pred_'+model[4]], on=['date', 'up_down'], how='inner')

output['sum'] =  output['dst'] + output['dst'] + output['logreg'] 

# Trade is executed only when any 2 models converge
a = output['up_down'][output['sum']>=2].count()  
b = output['up_down'][(output['sum']>=2) & (output['up_down']==1)].count()

c = output['up_down'][output['sum']<=1].count()
d = output['up_down'][(output['sum']<=1) & (output['up_down']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(pred_dict['pred_'+model[0]]), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', (b+d)/(a+c)*100,'%')

print('UP Accuracy:', (b)/(a)*100,'%')
print('DOWN Accuracy:', (d)/(c)*100,'%')

# Create the confusion matrix
confusion_matrix = np.array([[d, c-d], [a-b, b]])

print("Confusion Matrix:")
print(confusion_matrix)

Predictions Window-size: 1507 
 Total Predictions Made: 1507 
 Correct Predictions: 799
Overall Accuracy: 53.01924353019244 %
UP Accuracy: 57.42705570291777 %
DOWN Accuracy: 48.60557768924303 %
Confusion Matrix:
[[366 387]
 [321 433]]
